In [49]:
import pandas as pd

In [ ]:
data = pd.read_csv("vehicles.csv", header = None, skiprows=[0])
column_headers = pd.read_csv("vehicles.csv", nrows=0).columns.tolist()
data.columns = column_headers
data = data.drop(["id", "url", "region_url", "VIN", "size", "image_url", "description", "county", "lat", "long"], axis=1)

print(data.isna().sum())
#id, url, region_url, VIN, size, image_url, description, county, lat, long
#print(data.posting_date.value_counts())
#print(data.shape[0])
display(data)


region               0
price                0
year              1004
manufacturer     11321
model             3218
condition       106771
cylinders       108930
fuel              1750
odometer          2785
title_status      5420
transmission      1555
drive            83074
type             59274
paint_color      79944
state                0
posting_date        50
dtype: int64
posting_date
2021-04-13T13:19:15-0500    11
2021-04-22T20:32:05-0400    11
2021-04-29T20:06:09-0500     9
2021-04-14T22:44:45-0400     8
2021-05-03T20:46:03-0700     8
                            ..
2021-05-02T12:19:46-0400     1
2021-05-02T12:15:12-0400     1
2021-05-02T12:05:03-0400     1
2021-05-02T12:04:39-0400     1
2021-04-06T                  1
Name: count, Length: 243133, dtype: int64


,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color,state,posting_date
0,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN
1,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN
2,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN
3,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN
4,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263455,santa fe / taos,15000,1985.0,chevrolet,k10 4x4,NaN,NaN,gas,4500.0,clean,manual,NaN,NaN,grey,nm,2021-04-06T16:13:56-0600
263456,santa fe / taos,16500,2011.0,bmw,x3 xdrive35i,excellent,6 cylinders,gas,85000.0,lien,automatic,4wd,SUV,white,nm,2021-04-06T13:45:45-0600
263457,santa fe / taos,19990,2013.0,volkswagen,eos sport convertible,good,NaN,gas,24294.0,clean,automatic,fwd,convertible,blue,nm,2021-04-06T13:11:00-0600
263458,santa fe / taos,32590,2018.0,bmw,x3 xdrive30i sport utility,good,NaN,gas,28118.0,clean,automatic,NaN,other,black,nm,2021-04-06T12:41:14-0600


In [51]:
from sklearn.model_selection import train_test_split

y = data["price"]
y.columns = ["price"]
X = data.drop("price", axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [52]:
#code for analysis

In [53]:
#code for model

In [54]:
#code for validation